In [1]:
import numpy as np
from scipy import sparse
import sys
import os
import time

In [2]:
np.random.seed(0)

## Sparse .npz (compressed) takes the least disk space regardless of density
Uncompressed sparse matrices take more disk space when the matrices are very dense

In [ ]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    mat = np.load(f'data/{i}.npy')
    print(i, mat.shape)
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

    sparse_mat = sparse.csc_matrix(mat, copy=True)

    sparse.save_npz(f'data/{i}_sparse', sparse_mat)
    sparse.save_npz(f'data/{i}_sparse_un', sparse_mat, compressed=False)
    
    dense_size = os.path.getsize(f'data/{i}.npy') / 1000000
    sparse_size = os.path.getsize(f'data/{i}_sparse.npz') / 1000000
    sparse_un_size = os.path.getsize(f'data/{i}_sparse_un.npz') / 1000000
    print(f'dense: {dense_size :.4} MB\nsparse: {sparse_size :.4} MB\nsparse_uncompressed: {sparse_un_size :.4} MB\n')

CCpGeAeGaD (1391, 137)
density: 97.8 %
dense: 1.525 MB
sparse: 1.347 MB
sparse_uncompressed: 2.238 MB

DaGpBP (137, 11381)
density: 21.0 %
dense: 12.47 MB
sparse: 2.529 MB
sparse_uncompressed: 3.982 MB

MFpGdCcSE (2884, 5734)
density: 14.1 %
dense: 132.3 MB
sparse: 17.79 MB
sparse_uncompressed: 27.99 MB

GiGpBP (20945, 11381)
density: 4.65 %


## Memory-map reduces matrix-vector multiplication time for lower-density matrices

Putting a normal load above a memmap load decreases memmap load time. However, loading memmap first does not speed up normal load times. Note, this only appears to be the case for very dense matrices. For best comparisons, we should always load a memmap matrix first in the sequence.

In [ ]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}.npy')
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

#     Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
#     4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode=None)
    output = vec @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    
#     Second memory-map load time for comparison
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
      
    print(f'Normal: {time_1 :.3} sec\nMMAP: {time_2 :.3} sec\nMMAP2: {time_3 :.3} sec\n')

## Sparse (uncompressed/compressed) vs Dense

Uncompressed sparse matrices load and multiply about as quickly as dense matrices. The order of matrix reads does not change for sparse reads from npz files. Compressed sparse matrices take an order of magnitude more time to load and multiply.

In [ ]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}.npy')
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

#     Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
#     4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy')
    output = vec @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = sparse.load_npz(f'data/{i}_sparse.npz')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = sparse.load_npz(f'data/{i}_sparse_un.npz')
    output = vec @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
    
    print(f'Dense No Memmap: {time_1 :.3} sec\nCompressed Sparse: {time_2 :.3} sec\nUncompressed Sparse: {time_3 :.3} sec\n')

## Subset matrix memmap based on nonzero rows of the search vector

In [ ]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}.npy')
    print(mat.shape)
    
#     Create a vector to multiply
    vector_size = mat.shape[0]
    vec4 = np.zeros((1, vector_size)).flatten()
    vec10 = np.zeros((1, vector_size)).flatten()
    
#     4 search nodes
    indices4 = np.random.randint(0, high=vector_size, size=4)
    indices10 = np.random.randint(0, high=vector_size, size=10)
    vec4[indices4] = 1
    vec10[indices10] = 1
    del mat
    print_str = ''
    
#     Simple memmap
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec4 @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    print_str += f'memmap4: {time_1 :.3}\n'
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec10 @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    print_str += f'memmap10: {time_2 :.3}\n'
    
#     No memmap
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode=None)
    output = vec4 @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
    print_str += f'no memmap4: {time_3 :.3}\n'
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode=None)
    output = vec10 @ matrix
    t2 = time.time()
    time_4 = t2 - t1
    del matrix
    del output
    print_str += f'no memmap10: {time_4 :.3}\n'
       
#     create new vector of ones
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = np.ones(4) @ matrix[indices4]
    t2 = time.time()
    time_5 = t2 - t1
    del matrix
    del output
    print_str += f'subset 4: {time_5 :.3}\n'
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = np.ones(10) @ matrix[indices10]
    t2 = time.time()
    time_6 = t2 - t1
    del matrix
    del output
    print_str += f'subset 10: {time_6 :.3}\n'
        
    print(print_str)